In [1]:
from braindecode.datasets.moabb import MOABBDataset

subject_id = [1,2,3,4,5,6,7,8,9]
#subject_id = [1]
#2a= BNCI2014001 2b= BNCI2014004
dataset = MOABBDataset(dataset_name="BNCI2014001", subject_ids=subject_id)
print(dataset)


SHA256 hash of downloaded file: 15850d81b95fc88cc8b9589eb9b713d49fa071e28adaf32d675b3eaa30591d6e
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 81916dff2c12997974ba50ffc311da006ea66e525010d010765f0047e771c86a
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 77387d3b669f4ed9a7c1dac4dcba4c2c40c8910bae20fb961bb7cf5a94912950
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 8b357470865610c28b2f1d351beac247a56a856f02b2859d650736eb2ef77808
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 4dc3

48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
4

In [2]:
dataset.datasets[4]
dataset.save("dataset\\2amoabb\\train")

Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\0-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\0-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\1-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\1-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\2-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\2-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\3-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\3-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\4-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2amoabb\train\4-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motorimagery_fo

Opening raw data file <braindecode.datasets.moabb.MOABBDataset object at 0x0000018289D56940>...


TypeError: fname must be an instance of path-like, got <class 'braindecode.datasets.moabb.MOABBDataset'> instead.